# 🎙️ Entrenamiento de Voz Piper TTS en Google ColabEste notebook permite entrenar modelos de voz Piper TTS con GPU gratuita de Google Colab.**GPU recomendada:** T4 o superior  **Tiempo por época:** ~10-30 minutos con GPU (vs 30-60 min en CPU)---

## 1️⃣ Configuración InicialVerificar GPU y montar Google Drive

In [ ]:
# Verificar GPU disponible!nvidia-smiimport torchprint(f"\n✅ PyTorch version: {torch.__version__}")print(f"✅ CUDA available: {torch.cuda.is_available()}")if torch.cuda.is_available():    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

In [ ]:
# Montar Google Drivefrom google.colab import drivedrive.mount('/content/drive')# Crear directorio de trabajo!mkdir -p /content/piper-training%cd /content/piper-training

## 2️⃣ Instalación de DependenciasInstalar Piper y todas las dependencias

In [ ]:
%%bashset -eecho "📦 Instalando dependencias del sistema..."apt-get update -qqapt-get install -y -qq espeak-ng wget git > /dev/null 2>&1echo "✅ espeak-ng instalado"espeak-ng --version

In [ ]:
%%bashset -eecho "📦 Instalando piper-phonemize..."if [ ! -d "piper_phonemize" ]; then    wget -q https://github.com/rhasspy/piper-phonemize/releases/download/v1.2.0/piper_phonemize-amd64.tar.gz    tar -xzf piper_phonemize-amd64.tar.gz    rm piper_phonemize-amd64.tar.gz    echo "✅ piper-phonemize instalado"else    echo "✅ piper-phonemize ya existe"fi

In [ ]:
%%bashset -eecho "📦 Clonando Piper..."if [ ! -d "piper" ]; then    git clone -q https://github.com/rhasspy/piper.git    echo "✅ Piper clonado"else    echo "✅ Piper ya existe"fi

In [ ]:
# Instalar dependencias Pythonprint("📦 Instalando PyTorch y dependencias...")!pip install -q torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121!pip install -q "numpy>=1.26,<2.0"!pip install -q "pytorch-lightning>=1.9.0,<2.0.0"!pip install -q librosa onnxruntime scipy cython%cd piper/src/python!pip install -q -e . --no-deps%cd /content/piper-trainingprint("✅ Dependencias instaladas")

## 3️⃣ Descargar Modelo Base

In [ ]:
%%bashset -emkdir -p models_base && cd models_baseif [ ! -f "en_US-lessac-high.ckpt" ]; then    echo "📥 Descargando checkpoint (952 MB)..."    wget -q --show-progress \        "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/7bf647cb000d8c8319c6cdd4289dd6b7d0d3eeb8/en/en_US/lessac/high/epoch=2218-step=838782.ckpt" \        -O en_US-lessac-high.ckpt    echo "✅ Checkpoint descargado"else    echo "✅ Checkpoint ya existe"fi

## 4️⃣ Configurar DatasetSube tu dataset preprocesado o procesa uno nuevo

In [ ]:
# OPCIÓN A: Copiar desde Google DriveDRIVE_PATH = "/content/drive/MyDrive/piper-datasets/sig"  # ⬅️ AJUSTAR!mkdir -p datasets!cp -r "{DRIVE_PATH}" datasets/import osDATASET_NAME = os.path.basename(DRIVE_PATH)DATASET_DIR = f"datasets/{DATASET_NAME}"print(f"✅ Dataset copiado: {DATASET_DIR}")

In [ ]:
# OPCIÓN B: Subir ZIP y preprocesarfrom google.colab import filesuploaded = files.upload()  # Sube tu dataset.zip aquí

## 5️⃣ Entrenar Modelo

In [ ]:
# Configuración de entrenamientoDATASET_DIR = "datasets/sig"  # ⬅️ AJUSTARMAX_EPOCHS = 100BATCH_SIZE = 16  # Ajustar según VRAMCHECKPOINT_EPOCHS = 5!python -m piper_train \  --dataset-dir {DATASET_DIR} \  --accelerator gpu \  --devices 1 \  --batch-size {BATCH_SIZE} \  --validation-split 0.05 \  --num-test-examples 0 \  --max_epochs {MAX_EPOCHS} \  --quality high \  --resume_from_checkpoint models_base/en_US-lessac-high.ckpt \  --checkpoint-epochs {CHECKPOINT_EPOCHS} \  --precision 32

## 6️⃣ Monitorear Progreso

In [ ]:
# Ver métricasimport pandas as pdimport matplotlib.pyplot as pltfrom pathlib import Pathlogs_dir = Path(DATASET_DIR) / "lightning_logs"versions = sorted(logs_dir.glob("version_*"))if versions:    metrics = versions[-1] / "metrics.csv"    if metrics.exists():        df = pd.read_csv(metrics)        print(df.tail())                plt.figure(figsize=(12, 4))        plt.subplot(1, 2, 1)        plt.plot(df['epoch'], df['loss_gen_all'])        plt.title('Generator Loss')        plt.subplot(1, 2, 2)        plt.plot(df['epoch'], df['loss_disc_all'])        plt.title('Discriminator Loss')        plt.tight_layout()        plt.show()

## 7️⃣ Exportar Modelo

In [ ]:
# Exportar a ONNXfrom pathlib import Pathlogs_dir = Path(DATASET_DIR) / "lightning_logs"versions = sorted(logs_dir.glob("version_*"))checkpoints = sorted(versions[-1].glob("checkpoints/*.ckpt"))CHECKPOINT = str(checkpoints[-1])!mkdir -p outputs!cd piper/src/python && python3 -m piper_train.export_onnx \    "{CHECKPOINT}" \    "/content/piper-training/outputs/model.onnx"!cp {DATASET_DIR}/config.json outputs/model.onnx.jsonprint("✅ Modelo exportado a outputs/")

## 8️⃣ Descargar Resultados

In [ ]:
# Guardar en Google Drive!mkdir -p "/content/drive/MyDrive/piper-models/trained_model"!cp -r outputs/* "/content/drive/MyDrive/piper-models/trained_model/"print("✅ Guardado en Drive")

In [ ]:
# O descargar directamentefrom google.colab import files!zip -r model_trained.zip outputs/files.download("model_trained.zip")